Import packages and setting parameters

In [3]:
# packages
import numpy  as np 
import pandas as pd
import os
pd.set_option("display.max_columns", None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# --- logging --- #
import logging
logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

# --- parameters --- #
# path tho the main table saved as csv
table_path = '/fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/xml_table_from_drive.csv'

# ------------------ #

logger.info(f'Taking the .csv file from {table_path}')

Taking the .csv file from /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/xml_table_from_drive.csv
Taking the .csv file from /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/xml_table_from_drive.csv


# Reading the `.csv` dataset

You need to download the table at  [Common MAGIC LST1 data - auto](https://docs.google.com/spreadsheets/d/1Tya0tlK-3fuN6_vXOU5FwJruis5vBA9FALAPLFHOfBQ/edit) and upload it

In [11]:
# column names of the table
vnames = ['Date', 'Source', 'Conditions', 'Trigger', 'Stereo', 'HV', 'MinZd [deg]', 'MaxZd [deg]',
          'LST runs', 'MAGIC first run', 'MAGIC last run', 'Obs time (min)', 'Same wobble (min)', '-']

# reading the table
logger.info(f'Reading .csv file from {table_fname}')
df = pd.read_csv(table_path, names=vnames, index_col=False).drop('-', axis=1)
df = df[df['Source'].notna()][1:].reset_index(drop=True)

# now we convert magic runs to an array of runs
df['MAGIC runs'] = [np.arange(int(df['MAGIC first run'][n]), int(df['MAGIC last run'][n]) + 1, 1) for n in range(len(df))]    
df['LST runs']   = [np.fromstring(df['LST runs'][n][1:-1], dtype='int', sep=',')                  for n in range(len(df))] 
df.drop(['MAGIC first run', 'MAGIC last run'], axis=1, inplace=True)

# putting together LST and MAGIC run columns, interchanging places
df['Obs time (min)'], df.loc[:,'MAGIC runs'] = df['MAGIC runs'], df['Obs time (min)']
df.rename(columns={'Obs time (min)':'MAGIC runs','MAGIC runs':'Obs time (min)'}, inplace=True)

# updating some of the columns
for string in ['MinZd [deg]', 'MaxZd [deg]', 'Obs time (min)', 'Same wobble (min)']:
    df[string] = pd.to_numeric(df[string])

# adding a date column
df['Date'] = pd.to_datetime(df['Date'], format='%Y_%m_%d')   
    
display(df)

Reading .csv file from /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/xml_table_from_drive.csv
Reading .csv file from /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/xml_table_from_drive.csv


,Date,Source,Conditions,Trigger,Stereo,HV,MinZd [deg],MaxZd [deg],LST runs,MAGIC runs,Same wobble (min),Obs time (min)
0,2019-11-23,CrabNebula,['Dark'],['Sum'],[True],['Nominal'],9.82,15.38,[1622],"[5086812, 5086813]",0.00,29.66
1,2019-11-24,CrabNebula,['Dark'],['Sum'],[True],['Nominal'],8.26,14.49,"[1631, 1632, 1633]","[5086858, 5086859]",0.00,31.28
2,2019-11-26,CrabNebula,['Dark'],['L3T'],[True],['Nominal'],9.92,23.16,"[1651, 1652, 1653]","[5086952, 5086953]",0.00,45.12
3,2020-01-14,CrabNebula,['Dark'],['SumT'],[True],['Nominal'],17.39,29.16,"[1787, 1788]","[5088351, 5088352, 5088353]",0.00,58.14
4,2020-01-15,CrabNebula,"['Dark', 'Moon']","['SumT', 'L3T']",[True],['Nominal'],19.18,42.52,"[1799, 1800, 1801]","[5088408, 5088409, 5088410, 5088411, 5088412, ...",0.00,77.78
...,...,...,...,...,...,...,...,...,...,...,...,...
399,2022-11-19,FRB20220912A,['Dark'],['L1_M2'],[False],['Nominal'],27.73,36.97,[10868],"[2006669, 2006670, 2006671, 2006672]",10.28,63.06
400,2022-11-20,Boomerang-Ta,['Dark'],['L1_M2'],[False],['Nominal'],62.51,69.86,"[10905, 10906, 10907]","[2006706, 2006707, 2006708, 2006709, 2006710]",32.94,61.91
401,2022-11-20,CrabNebula,"['Dark', 'Moon']",['L1_M2'],[False],['Nominal'],6.97,47.73,"[10909, 10910, 10911, 10912, 10913, 10914, 109...","[2006713, 2006714, 2006715, 2006716, 2006717, ...",186.17,214.66
402,2022-11-20,FRB20220912A,['Dark'],['L1_M2'],[False],['Nominal'],35.83,57.31,[10904],"[2006697, 2006698, 2006699, 2006700, 2006701, ...",126.71,131.99


# Filtering conditions
For the different sources we want to study, in our case:
- **Crab Nebula**
- **BL Lac**

In [21]:
# -- parameters -- #
source_names        = ['CrabNebula', 'BLLac']
source_common_names = ['Crab', 'BLLac']
min_same_wobble = 0
max_date        = '2022-05-01'

# filtering the dataset, the source, the wobble and the date
dfs = []
for source in source_names:
    logger.info(f'Filtering dataset with:\n--> Source {source}\n--> Minimum same wobble per night {min_same_wobble}\n--> Maximum date {max_date}')
    df_tmp = df[(df['Source'] == source_name) & (df['Same wobble (min)'] >= min_same_wobble) & (df['Date'] < max_date)]
    df_tmp = dfCrab.reset_index(drop=True)
    dfs.append(df_tmp)
    
    display(df_tmp.head(5))

Filtering dataset with:
--> Source CrabNebula
--> Minimum same wobble per night 0
--> Maximum date 2022-05-01
Filtering dataset with:
--> Source CrabNebula
--> Minimum same wobble per night 0
--> Maximum date 2022-05-01


,Date,Source,Conditions,Trigger,Stereo,HV,MinZd [deg],MaxZd [deg],LST runs,MAGIC runs,Same wobble (min),Obs time (min)
0,2019-11-23,CrabNebula,['Dark'],['Sum'],[True],['Nominal'],9.82,15.38,[1622],"[5086812, 5086813]",0.0,29.66
1,2019-11-24,CrabNebula,['Dark'],['Sum'],[True],['Nominal'],8.26,14.49,"[1631, 1632, 1633]","[5086858, 5086859]",0.0,31.28
2,2019-11-26,CrabNebula,['Dark'],['L3T'],[True],['Nominal'],9.92,23.16,"[1651, 1652, 1653]","[5086952, 5086953]",0.0,45.12
3,2020-01-14,CrabNebula,['Dark'],['SumT'],[True],['Nominal'],17.39,29.16,"[1787, 1788]","[5088351, 5088352, 5088353]",0.0,58.14
4,2020-01-15,CrabNebula,"['Dark', 'Moon']","['SumT', 'L3T']",[True],['Nominal'],19.18,42.52,"[1799, 1800, 1801]","[5088408, 5088409, 5088410, 5088411, 5088412, ...",0.0,77.78


Filtering dataset with:
--> Source BLLac
--> Minimum same wobble per night 0
--> Maximum date 2022-05-01
Filtering dataset with:
--> Source BLLac
--> Minimum same wobble per night 0
--> Maximum date 2022-05-01


,Date,Source,Conditions,Trigger,Stereo,HV,MinZd [deg],MaxZd [deg],LST runs,MAGIC runs,Same wobble (min),Obs time (min)
0,2019-11-23,CrabNebula,['Dark'],['Sum'],[True],['Nominal'],9.82,15.38,[1622],"[5086812, 5086813]",0.0,29.66
1,2019-11-24,CrabNebula,['Dark'],['Sum'],[True],['Nominal'],8.26,14.49,"[1631, 1632, 1633]","[5086858, 5086859]",0.0,31.28
2,2019-11-26,CrabNebula,['Dark'],['L3T'],[True],['Nominal'],9.92,23.16,"[1651, 1652, 1653]","[5086952, 5086953]",0.0,45.12
3,2020-01-14,CrabNebula,['Dark'],['SumT'],[True],['Nominal'],17.39,29.16,"[1787, 1788]","[5088351, 5088352, 5088353]",0.0,58.14
4,2020-01-15,CrabNebula,"['Dark', 'Moon']","['SumT', 'L3T']",[True],['Nominal'],19.18,42.52,"[1799, 1800, 1801]","[5088408, 5088409, 5088410, 5088411, 5088412, ...",0.0,77.78


# Generate MAGIC runs `.txt` to launch the slurm job calib $\rightarrow$ dl1

In [22]:
magic_paths  = [f'/fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/MAGIC_runs{source}.txt' for source in source_common_names]

for dframe, path in zip(dfs, magic_paths):
    # generating the text files for MAGIC runs
    # adding an upper label
    stringM = '# MAGIC runs (separed by commas for each night)'
    
    # adding for each night
    for n in range(len(dframe)):
        str_n   = ','.join(dframe['MAGIC runs'][n].astype(str))
        stringM = f'{stringM}\n# Night {n}\n{str_n}'

    with open(path, 'w') as f:
        f.write(stringM)

    # print(stringM)
    logger.info(f'File written: {path}\n')

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/MAGIC_runsCrab.txt

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/MAGIC_runsCrab.txt

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/MAGIC_runsBLLac.txt

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/MAGIC_runsBLLac.txt



# Generate LST-MAGIC runs `.txt` to launch the slurm job dl1 $\rightarrow$ stereo
Will be the same `.txt` needed to launch the stereo $\rightarrow$ dl2

In [25]:
common_paths  = [f'/fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_data{source}.txt' for source in source_common_names]

for dframe, path in zip(dfs, common_paths):
    
    # generating the text files for LST+MAGIC for each LST subrun repeating row
    # creating a label
    string = '#LST runs - MAGIC runs (separed by commas)'
    
    for n in range(len(dframe['LST runs'])):
        for s in range(len(dframe['LST runs'][n])):
            LSTsub, MRuns = dframe['LST runs'][n][s], ','.join(dframe['MAGIC runs'][n].astype(str))
            str_n  = f'{LSTsub}-{MRuns}'
            if s == 0:
                string = f'{string}\n# Night {n}\n{str_n}'
            else:
                string = f'{string}\n{str_n}'

    with open(path, 'w') as f:
        f.write(string)
        
    # print(string)
    logger.info(f'File written: {path}\n')

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_dataCrab.txt

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_dataCrab.txt

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_dataBLLac.txt

File written: /fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_dataBLLac.txt



# Statistics
We can extract some statistics from the common data `.csv` file

In [26]:
aa = df.loc[:232,].groupby(['Source']).sum().sort_values('Obs time (min)').tail(10)
aa['Obs time (hour)'] = aa['Obs time (min)']/60
display(aa)

/tmp/ipykernel_9312/2776046190.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aa = df.loc[:232,].groupby(['Source']).sum().sort_values('Obs time (min)').tail(10)


,MinZd [deg],MaxZd [deg],Same wobble (min),Obs time (min),Obs time (hour)
Source,,,,,
1A-0535+262,48.36,97.80,8.08,330.94,5.515667
BLLac,304.19,369.16,237.65,366.59,6.109833
1ES1959+650,287.45,324.51,289.90,515.32,8.588667
RSOph,266.18,368.66,616.42,656.65,10.944167
M87,222.26,332.76,593.72,736.31,12.271833
Mrk421,426.10,643.61,592.28,1030.41,17.173500
GalacticCent,1347.45,1423.54,844.06,1084.89,18.081500
Mrk501,652.80,980.27,1295.70,1613.05,26.884167
LHAASOJ2108+,623.31,897.59,1660.42,1936.12,32.268667
